In [1]:
%matplotlib inline 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error as mae

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import scale

from IPython.display import clear_output

import structural_model

In [2]:
before = pd.read_csv('./input/prepared_data/test_before_proc.csv')
before = before.rename(columns={'Скважина' : 'well',
                                'Дата' : 'date',
                                'Нефть, т' : 'prod',
                                'Закачка, м3' : 'inj'})
before = before[['well', 'date', 'prod', 'inj', 'X', 'Y', 'group']]

after = pd.read_csv('./input/prepared_data/test_after_proc.csv')
after = after.rename(columns={'Скважина' : 'well',
                                'Дата' : 'date',
                                'Закачка, м3' : 'inj'})
after = after[['well', 'date', 'inj', 'group']]

In [3]:
coords = before.groupby('well')[['X', 'Y']].mean()
prod = before[['well', 'date', 'prod']].set_index(['well', 'date']).unstack(0)
prod.columns = prod.columns.droplevel()
inj = before[['well', 'date', 'inj']].set_index(['well', 'date']).unstack(0)
inj.columns = inj.columns.droplevel()
after_inj = after[['well', 'date', 'inj']].set_index(['well', 'date']).unstack(0)
after_inj.columns = after_inj.columns.droplevel()

In [4]:
groups = {}
for g in before.group.unique():
    groups[g] = before[before.group == g].well.unique()

In [9]:
scores = structural_model.model(prod, inj, groups, SVR, SVR, 
                                {'kernel' : 'linear', 'epsilon' : 5, 'C' : 100}, 
                                {'kernel' : 'linear', 'epsilon' : 8})

50


In [6]:
np.mean(scores)

236.11653938373868

In [7]:
np.mean([w for i, w in enumerate(scores) if i != 9])

140.47307791114855

In [8]:
predictions = structural_model.prediction(prod, inj, after_inj, groups, SVR, SVR,
                                         {'kernel' : 'linear', 'epsilon' : 5}, 
                                         {'kernel' : 'linear', 'epsilon' : 8})

50


In [10]:
preds = [s.to_dict('series') for s in list(predictions.values())]

npreds = {k : v for d in preds for k, v in d.items()}

order = pd.read_csv('./input/order3.csv')

p = []
for well in order.well:
    p.append(npreds[well].values.tolist())

from itertools import chain

p = pd.DataFrame(list(chain(*p)))

p.to_csv('./lol.csv')

(p > 0).all()

0    True
dtype: bool